In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

# 4.1 FewShotPromptTemplate

In [2]:
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [3]:
t = PromptTemplate.from_template("What is the capital of {country}?")
t

PromptTemplate(input_variables=['country'], template='What is the capital of {country}?')

In [4]:
t.format(country="Spain")

'What is the capital of Spain?'

In [5]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

In [6]:
chat.predict("What do you know about Greece?")

Greece is a country located in southeastern Europe, known for its rich history, cultural heritage, and stunning landscapes. Here are some key points about Greece:

1. **Geography**: Greece is situated on the southern tip of the Balkan Peninsula and is bordered by Albania, North Macedonia, Bulgaria, and Turkey. It has a vast coastline along the Aegean, Ionian, and Mediterranean Seas, and is made up of numerous islands, with Crete being the largest.

2. **History**: Greece is often referred to as the cradle of Western civilization. It is the birthplace of democracy, philosophy, and the Olympic Games. Ancient Greece was home to great philosophers such as Socrates, Plato, and Aristotle, as well as significant historical figures like Alexander the Great.

3. **Culture**: Greek culture is characterized by its contributions to art, literature, and science. The ancient Greeks made significant advancements in various fields, including mathematics, astronomy, and medicine. Greek mythology, with 

"Greece is a country located in southeastern Europe, known for its rich history, cultural heritage, and stunning landscapes. Here are some key points about Greece:\n\n1. **Geography**: Greece is situated on the southern tip of the Balkan Peninsula and is bordered by Albania, North Macedonia, Bulgaria, and Turkey. It has a vast coastline along the Aegean, Ionian, and Mediterranean Seas, and is made up of numerous islands, with Crete being the largest.\n\n2. **History**: Greece is often referred to as the cradle of Western civilization. It is the birthplace of democracy, philosophy, and the Olympic Games. Ancient Greece was home to great philosophers such as Socrates, Plato, and Aristotle, as well as significant historical figures like Alexander the Great.\n\n3. **Culture**: Greek culture is characterized by its contributions to art, literature, and science. The ancient Greeks made significant advancements in various fields, including mathematics, astronomy, and medicine. Greek mythology

In [7]:
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

In [8]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt

FewShotPromptTemplate(input_variables=['country'], examples=[{'question': 'What do you know about France?', 'answer': '\n            Here is what I know:\n            Capital: Paris\n            Language: French\n            Food: Wine and Cheese\n            Currency: Euro\n            '}, {'question': 'What do you know about Italy?', 'answer': '\n            I know this:\n            Capital: Rome\n            Language: Italian\n            Food: Pizza and Pasta\n            Currency: Euro\n            '}, {'question': 'What do you know about Greece?', 'answer': '\n            I know this:\n            Capital: Athens\n            Language: Greek\n            Food: Souvlaki and Feta Cheese\n            Currency: Euro\n            '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], template='\n    Human: {question}\n    AI: {answer}\n'), suffix='Human: What do you know about {country}?')

In [9]:
prompt.format(country="Germany")

'\n    Human: What do you know about France?\n    AI: \n            Here is what I know:\n            Capital: Paris\n            Language: French\n            Food: Wine and Cheese\n            Currency: Euro\n            \n\n\n\n    Human: What do you know about Italy?\n    AI: \n            I know this:\n            Capital: Rome\n            Language: Italian\n            Food: Pizza and Pasta\n            Currency: Euro\n            \n\n\n\n    Human: What do you know about Greece?\n    AI: \n            I know this:\n            Capital: Athens\n            Language: Greek\n            Food: Souvlaki and Feta Cheese\n            Currency: Euro\n            \n\n\nHuman: What do you know about Germany?'

In [10]:
chain = prompt | chat

chain.invoke({"country": "Germnay"})

AI: 
            I know this:
            Capital: Berlin
            Language: German
            Food: Sausages and Sauerkraut
            Currency: Euro

AIMessageChunk(content='AI: \n            I know this:\n            Capital: Berlin\n            Language: German\n            Food: Sausages and Sauerkraut\n            Currency: Euro')

# 4.2 FewShowChatMessagePromptTemplate

In [11]:
examples = [
    {
        "country": "France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
    },
    {
        "country": "Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
    },
    {
        "country": "Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

fs_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. Youu give short answers."),
        fs_example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Republic of Korea"})


            I know this:
            Capital: Seoul
            Language: Korean
            Food: Kimchi and Bibimbap
            Currency: South Korean Won (KRW)
            

AIMessageChunk(content='\n            I know this:\n            Capital: Seoul\n            Language: Korean\n            Food: Kimchi and Bibimbap\n            Currency: South Korean Won (KRW)\n            ')

# 4.3 LengthBasedExampleSelector

In [18]:
from typing import Any, Dict


class RandomExapleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        return super().add_example(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [26]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     examples=examples, example_prompt=example_prompt, max_length=120
# )

example_selector = RandomExapleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert"),
#     fs_example_prompt,
#     ("human", "What do you know about {country}?")
# ])


# chain = final_prompt | chat

# chain.invoke({"country": "Republic of Korea"})

'\n    Human: What do you know about Italy?\n    AI: \n            I know this:\n            Capital: Rome\n            Language: Italian\n            Food: Pizza and Pasta\n            Currency: Euro\n            \n\n\nHuman: What do you know about Brazil?'

# 4.4 Serialization and Composition

In [30]:
# load prompt in json format.

prompt_json = load_prompt("./prompt.json")

prompt_json.format(country="South Korea")

'What is the capital of South Korea'

In [32]:
# load prompt in yaml format.

prompt_yaml = load_prompt("./prompt.json")

prompt_yaml.format(country="France")

'What is the capital of France'

In [34]:
# pipeline

intro = PromptTemplate.from_template(
    """
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
    """
{intro}

{example}

{start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

full_prompt.format(
    character="Assassin",
    example_question="What is your location",
    example_answer="Shhhh, That is secret",
    question="Who is your target?",
)

'\n\nYou are a role playing assistant.\nAnd you are impersonating a Assassin\n\n\n\nThis is an example of how you talk:\n\nHuman: What is your location\nYou: Shhhh, That is secret\n\n\n\nStart now!\n\nHuman: Who is your target?\nYou:\n\n'

In [35]:
chain = full_prompt | chat

chain.invoke(
    {
        "character": "Assassin",
        "example_question": "What is your location",
        "example_answer": "Shhhh, That is secret",
        "question": "Who is your target?",
    }
)

Shhhh, that information is classified. Discretion is key in my line of work.

AIMessageChunk(content='Shhhh, that information is classified. Discretion is key in my line of work.')